# Exercise Group - Chapter 2

Below you find exercises for chapter 2. These exercises will prepare you in particular for the obligatory mid-term tests. The purpose of the prepared code snippets below is to give you some helpful structure for coding by predefining some useful variable names, but you do not need to use these prepared code snippets and prepared variable names if you do not want to. Do not hestitate to use the function `help()` to print the documentation of the functions that you need. If you are stuck, please ask one of the supervisors present.

In the shared folder you will find a subfolder `data` containing important time series. You may copy them to your "*my_materials*" folder in order to import the data from there if you like.

<br><br>

# Topic 1: Historical Simulation

In the directory `data` you will find a file named `DAX_index.csv`. It contains the daily closing levels $V_{n}$, $n \in \{0, \dots, N\}$, of the German DAX index between 03 January 2000 and 11 October 2024, making up for a total of $N + 1 = 6310$ data points. In this exercise we want to calculate the VaR estimates for the DAX timeseries using the method of historical simulation as outlined in Section 2.3. Additionally, the goal is to obtain confidence intervals for the VaR estimates using the method in Section 2.3.2.. 

<br>

## Step 1: Compute the Value-at-Risk Estimates

Compute the VaR estimates at level $\alpha = 0.95$ for each trading day after the first trading year using $L_{n + 1} = -\big(V_{n + 1} - V_n\big) = V_n (1 - \exp(X_{n+1}))$ as losses, where $(X_n)_{n \in \{1, \dots, N\}}$ are the log-returns.

**Task 1.1**: Define an array `L` that contains the losses. Then, define an array `L_windows` of shape `(N - 252, 252)` that contains windows of length 252 of the DAX time series. That is, the `i`th row of `L_windows` should contain the array `L[i:(i + 252)]`.

**Task 1.2**: Our goal is now to compute the 0.95-VaR estimate $\widehat{\mathrm{VaR}_{0.95}}(L_{n+1})$ using the method of historical simulation outlined in Section 2.3.3 based on the previous $252$ observations. That is, for each $n \in \{252, \dots, N\}$, we want to use the past year of losses $L_{i}$, $i \in \{n-251, \dots n\}$, in order to obtain the empirical 0.95-quantile of the conditional law of $L_{n + 1}$. To do so, write a function `value_at_risk(windows, alpha)` that takes an array `windows` of shape `(M, 252)` as an input and computes the empirical `alpha`-quantile of each row. The output of your function should have shape `(M,)` and your function should contain no `for`-loops. Then store the output of your function applied to the array `L_windows` and `alpha=0.95` in an array `VaR`.

<br>

## Step 2: Compute Confidence Intervals for the Value-at-Risk

We would now like to obtain a confidence interval at level $p = 0.99$ for each of the Value-at-Risk estimates stored in the array `VaR` using the approach from Section 2.3.2. 

**Task 1.3**: Compute the integer $i$ from equation (2.6) in the lecture notes. That is, the integer $i$ is given by $1$ plus the $\frac{1 + p}{2} = 0.995$-quantile of the binomial distribution with parameters $m = 252$ and $1 - \alpha = 0.05$.

<details>
  <summary>Result:</summary>
    
The integer $i$ is given by $i = 23$.

</details>

**Task 1.4**: Compute the integer $j$ from equation (2.7) in the lecture notes. That is, the integer $j$ is given by the $\frac{1 - p}{2} = 0.005$-quantile of the binomial distribution with parameters $m = 252$ and $1 - \alpha = 0.05$.

<details>
  <summary>Result:</summary>
    
The integer $j$ is given by $j = 5$.
</details>

**Task 1.5**: For each $n \in \{1, \dots, N-1\}$, compute the lower confidence boundary $X_{i:n}$ as well as the upper confidence boundary $X_{j:n}$ (see Section 2.3.2) using the corresponding integers $i$ and $j$ computed in Task 1.3 and 1.4, respectively. Store the lower and upper confidence boundaries in arrays `VaR_lower` and `Var_upper`.

<br>

## Step 3: Plot the VaR Estimates and Interpret the Results


**Task 1.6** Plot the Value-at-Risk estimates stored in the array `VaR` as well as the lower and upper confidence bounds for the VaR stored in the arrays `VaR_lower` and `VaR_upper` in a common plot. What do you observe? Compare your output to the parametric Gaussian VaR estimates from Week 2, Task 1.2.

<details>
  <summary>Interpretation:</summary>
    
As expected, the 99%-confidence interval for the 95%-VaR forms a tunnel around the computed Value-at-Risk. Instead of the point estimates of $\widehat{\mathrm{VaR}_{0.95}}(L_{n+1})$ shown by the orange line, this tunnel gives us a region in which the 95%-VaR falls with a probability of 99%. This confidence region is completely distribution-free, i.e. we do not pose any specific assumptions on the underlying returns distribution, as e.g. a Gaussian assumption. The only implicit assumption on the returns dynamics that is present here is the assumption that returns behave roughly stationary over periods of 1 year, i.e. 252 trading days. Compared to the parametric Gaussian VaR estimate of Week 2, Task 1.2, it is visible that the non-parametric VaR based on the historical simulation better adapts to sudden changes in conditional volatility. This also results in a better backtesting performance, with roughly 5.54% exceedences of the non-parametric 95%-VaR compared to roughly 6.16% exceedences of the Gaussian 95%-VaR (see Week 2, Task 1.3).
</details>

<img src="./Images/Task3_1_6.png" width="450">

<br><br>

# Topic 2: The Empirical CDF and Bootstrapping the Expected Shortfall

The goal of this exercise is to visualize the convergence stated in Theorem 2.3 and to use the empirical cdf of a sample in order to construct nonparametric confidence intervals, in our case of the Expected Shortfall.

<br> 

## Step 1: Generate Samples of the Normal Distribution

**Task 2.1**: Generate samples of the Standard Normal distribution $N(0, 1)$ with 10, 100, 1000 and 10000 observations, respectively. Use the `numpy` random seed 10 for reproducibility.

<br>

## Step 2: Visualize the Glivenko-Cantelli Theorem

**Task 2.2**: In a single plot, draw the empirical cdfs of the samples you generated next to the true cdf of the $N(0, 1)$ distribution over the interval $x \in [-4, 4]$.

<details>
  <summary>Interpretation:</summary>
    The larger the sample size $n$ becomes, the better the empirical cdf of the sample approximates the true underlying standard normal cdf (drawn in purple). In the case of $n = 10000$ samples, the empirical cdf drawn in red is already only barely distinguishable from its theoretical counterpart. The Glivenko-Cantelli theorem (Theorem 2.3.1 in the lecture notes) tells us that empirical cdfs of iid random variables converge uniformly to their theoretical true cdf counterparts, which is well visible in this plot.

</details>

<img src="./Images/Task3_2_2.png" width="450">

<br>

## Step 3: Computing Confidence Intervals for the Expected Shortfall via Bootstrap Methods

We now want to plot the 0.95-Expected Shortfall $\widehat{\mathrm{ES}_{0.95}}(L_{n + 1})$ together with a 99%-confidence interval for it in the same fashion as for the Value-at-Risk in Task 1.6.

**Task 2.3**: We want to compute the 0.95-ES estimate $\widehat{\mathrm{ES}_{0.95}}(L_{n+1})$ using the method of historical simulation outlined in Section 2.3.3 based on the previous $252$ observations. That is, for each $n \in \{252, \dots, N\}$, we want to use the past year of losses $L_{i}$, $i \in \{n-251, \dots n\}$, in order to obtain the empirical 0.95-Expected Shortfall for the law of $L_{n + 1}$. To do so, write a function `expected_shortfall(windows, alpha)` that takes an array `windows` of shape `(M, 252)` as an input and computes the empirical `alpha`-Expected Shortfall as in Section 2.3.3. The output of your function should have shape `(M,)` and your function should contain no `for`-loops. Then store the output of your function applied to the array `L_windows` and `alpha=0.95` in an array `ES`.

A widely used method to compute nonparametric confidence intervals of some sample statistic like the Expected Shortfall is the *Bootstrap method*. Suppose you have an iid data sample $X = (X_1, \dots, X_N)$ and you would like to obtain a confidence interval at level $\beta$ for some statistic $T(X)$ (in our case, $T(X)$ is the 0.95-Expected Shortfall of the sample $X$). The Bootstrap method proceeds by pretending that the true cdf of the distribution of the $X_i$ is equal to the empirical cdf of your sample. Afterwards, the Bootstrap method simulates $B$ different samples $X^{(b)} = (X^{(b)}_1, \dots, X^{(b)}_N)$, $b \in \{1, \dots, B\}$ from the empirical cdf. These $B$ samples can be used to calculate $B$ realisations of your statistic, $T(X^{(1)}), \dots, T(X^{(b)})$. You can then obtain the boundaries of a two-sided confidence interval at level $\beta$ for your statistic by computing the empirical $\frac{1-\beta}{2}$ quantile and the empirical $\frac{1 + \beta}{2}$ quantile of the realisations $T(X^{(1)}), \dots, T(X^{(b)})$.

**Task 2.4**: Calculate a 99%-confidence interval via the Bootstrap method for each expected shortfall estimate stored in the array `ES`. Store the lower and upper confidence boundaries in arrays `ES_lower` and `ES_upper`. Your code should proceed in the following steps:
- Write a `for`-loop over the different 1-year-windows of losses stored as rows in the array `L_windows`
- For each window, simulate $B = 1000$ samples of length 252 from the empirical cdf of the window. *Hint*: Simulating a sample of length 252 from the empirical cdf of the window is equivalent to drawing 252 times from the window with replacement.
- Then, for each window, use your function `expected_shortfall` to simultatenously compute the empirical 0.95-expected shortfall of each of the $B = 1000$ different samples.
- Finally, fow each window, compute the empirical 0.005-quantile and the empirical 0.995-quantile if the 1000 different expected shortfall estimates to obtain a confidence interval at level 0.99.

**Task 2.5**: Similar to Task 1.6, plot the Expected Shortfall estimates stored in the array `ES` as well as the lower and upper confidence bounds for the ES computed in Task 2.4 in a common plot. What do you observe? Compare your output to the plot from Task 1.6.

<details>
  <summary>Interpretation:</summary>
    
Again, the 99%-confidence interval for the 95%-ES forms a tunnel around the computed Expected Shortfall. Compared to the confidence region for the 95%-VaR from Task 1.6. the confidence region for the 95%-VaR is much more rugged, which originates from the fact that in the Bootstrap method, confidence intervals are effectively computed via Monte Carlo methods. As in the case of the VaR confidence region in Task 1.6, the ES confidence region lies at some times roughly symmetrically around the computed ES, while at other times (e.g. during the early COVID-19 outbreak) it lies clearly asymmetrically around the ES estimate. This shows that not only the variance but also the skewness of the conditional returns distribution is varying over time.
</details>


<img src="./Images/Task3_2_5.png" width="450">

<br><br>

# Topic 3: Monte Carlo Methods

Suppose you want to invest at the current time $N$ in the DAX index $V$ over the time horizon of ten days. You would like to model negative (percentage) returns as a loss, i.e. $L_{N + 10} = -\big(\frac{V_{N + 10}}{V_N} - 1\big) = 1 - \exp(X_{N+10})$, where $(X_n)_{n \in \{1, \dots, N\}}$ are the log-returns. Your goal is to obtain estimates of the 10-day Value-at-Risk (VaR) and Expected Shortfall (ES) corresponding to this loss. The aim of this exercise is to obtain the level $\alpha = 0.95$ VaR and ES estimates $\widehat{\mathrm{VaR}_{0.95}}(L_{N + 10})$ and $\widehat{\mathrm{ES}_{0.95}}(L_{N + 10})$ by fitting a GARCH(1, 1) model to the logarithmic returns of the DAX and then using the Monte Carlo method from Section 2.5.

<br>

## Step 1: Estimating the GARCH(1, 1) Parameters using Maximum-Likelihood Estimation

The goal of this step is to estimate the parameters $\alpha_0, \alpha_1, \beta$ and $\sigma^2_0$ of the GARCH(1, 1) model using the method described in Example 2.7:

**Task 3.1**: Define a function `sigma_sq_from_data` that takes the log-returns $(X_n)_{n \in \{0, \dots, N\}}$ as an input variable `x`, which should be an array of length $N + 1$, followed by the parameters of the GARCH(1, 1) model. It returns the corresponding GARCH variance sequence $(\sigma^2_n)_{n \in \{0, \dots, N\}}$ computed by the recursion from Section 2.1.5.

**Task 3.2**: Define a function `neg_log_lik_GARCH_11` which takes the parameter vector $\theta = (\alpha_0, \alpha_1, \beta, \sigma^2_0)$ and the array `x` of log-returns as input variables and returns the negative joint log-likelihood corresponding to the GARCH(1, 1) model as in Section 2.4 and Example 2.7.

**Task 3.3**: Compute the log-returns of the DAX series and store them in an array `log_returns`. Then, define an initial guess of parameter values stored in an array `init` and then compute the Maximum-Likelihood estimates for the GARCH(1, 1) parameters by using the `scipy` function `minimize` on your function `neg_log_lik_GARCH_11`. Print your resulting parameter estimates.

<details>
  <summary>Result:</summary>
    
The estimated parameters are $\alpha_0 \approx 3.093 \cdot 10^{-6}$, $\alpha_1 \approx 0.10451$, $\beta \approx 0.88060$ and $\sigma^2_0 \approx 0.00095$. Note that the simple GARCH(1, 1) condition for covariance stationarity, i.e. the condition $\alpha_1 + \beta < 1$ is fulfilled in this case.
</details>

<br><br>

## Step 2: Simulate Future Trajectories of the Fitted GARCH(1, 1) Model

In order to estimate the VaR and the ES using the method described in Section 2.5.1, we need multiple samples of the losses $L_{N + 10}$ over the period of 10 days. To compute samples of $L_{N + 10}$, we first need multiple samples of the index level $V_{N + 10}$ in 10 days.

**Task 3.4**: Use the function `sigma_sq_from_data` from the task above to obtain the past GARCH variance sequence $(\sigma^2_n)_{n \in \{0, \dots, N\}}$. Use the DAX log-returns `log_returns` as well as the Maximum-Likelihood estimates `alpha_0`, `alpha_1`, `beta` and `sigma_sq_0` computed in Step 1 as inputs to this function. Store the output of this function in an array `sigma_sq_historical`.

**Task 3.5**: Simulate $m = 10\,000$ trajectories of length $k = 10$ of the GARCH(1, 1) variance $(\sigma^2_n)_{n \in \{N + 1, \dots, N + 10\}}$ as well as $m = 10\,000$ trajectories of corresponding GARCH(1, 1) log-returns process $(X_n)_{n \in \{N + 1, \dots, N + 10\}}$ according to the recursions given in Section 2.1.5 and store them in arrays `sim_sigma_sq` and `sim_garch_X` of shape `(m, k)` . Use the final values `sigma_sq_historical[-1]` and `log_returns[-1]` as initial values for the recursive computations of $(\sigma^2_n)_{n \in \{N + 1, \dots, N + 10\}}$ and $(X_n)_{n \in \{N + 1, \dots, N + 10\}}$. Use the command `numpy.random.seed(10)` for reproducibility.

<br><br>

## Step 3: Compute VaR and ES Estimates

Using the simulated GARCH trajectories, we are now able to compute the losses $L_{N + 10}$ over the period of 10 days.

**Task 3.6**: Define an array `loss` of length $m = 10\,000$ that contains the multiple simulated losses $L_{N + 10} = -\big(\frac{V_{N + 10}}{V_N} - 1\big) = 1 - \exp(X_{N+10})$. Define an additional array `loss_sorted` that contains the elements of `loss` in descending order.

**Task 3.7**: Compute and print the estimates $\widehat{\mathrm{VaR}_{0.95}}(L_{N + 10})$ and $\widehat{\mathrm{ES}_{0.95}}(L_{N + 10})$ using the Monte Carlo method from Section 2.5.1.

<details>
  <summary>Result:</summary>
    
The 0.95-VaR for the 10-day percentage loss is approximately 0.9175%. The 0.95-ES for the 10-day percentage loss is approximately 1.1713%.
</details>